In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import plotly.express as px
from statannotations.Annotator import Annotator
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import scipy
sys.path.append('/work/include/')
import stat_process
import importlib
importlib.reload(stat_process)

<module 'stat_process' from '/work/include/stat_process.py'>

In [2]:
df = pd.read_csv("/work/processed/cleaned_data.csv")
df

,Unnamed: 0,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,intention_post_score,intention_change,single_use_perception,animal,group_category,group_label,condition,sustainable_choices,non_sustainable_choices,sustainable_choices_ratio
0,0,2025-01-20 01:33:39,2025-01-20 02:08:14,0,*******,100,2075,1,2025-01-20 02:08:15,R_7ZNWQQgwCOZcotd,...,5.0,0.0,2.0,seahorse,treatment,Chatbot + Character,treatment,"[1, 5, 7, 9, 11, 13]",[],6.00
1,1,2025-01-20 12:23:57,2025-01-20 12:29:57,0,*******,100,360,1,2025-01-20 12:29:58,R_63dfdErmTnYP9rX,...,3.0,-1.0,3.5,beluga,treatment,Chatbot + Character,treatment,[7],[4],1.00
2,2,2025-01-20 15:19:53,2025-01-20 15:23:09,0,*******,100,196,1,2025-01-20 15:23:10,R_6EI5znfDhCu9O0N,...,3.0,0.0,2.0,beluga,control_scientific,Scientific Fact + Image,control,[1],[],1.00
3,3,2025-01-20 15:18:37,2025-01-20 15:23:19,0,*******,100,282,1,2025-01-20 15:23:20,R_6FIR6EYvvCWrp7P,...,3.5,0.0,3.0,jellyfish,control_scientific,Scientific Fact + Image,control,[5],[],1.00
4,4,2025-01-20 15:19:16,2025-01-20 15:23:29,0,*******,100,252,1,2025-01-20 15:23:30,R_773GBdeWwBXKBzX,...,4.0,0.0,3.0,beluga,control_personal,Personal Story + Character,control,[13],[],1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,896,2025-01-20 17:46:16,2025-01-20 17:52:24,0,*******,100,367,1,2025-01-20 17:52:24,R_5uR1Le4uABzActq,...,3.0,0.0,5.0,seahorse,control_personal,Personal Story + Character,control,[5],[],1.00
885,897,2025-01-20 17:40:42,2025-01-20 17:53:28,0,*******,100,766,1,2025-01-20 17:53:29,R_1BXf09kTVDpJklz,...,2.5,0.0,2.0,seahorse,treatment,Chatbot + Character,treatment,[],[4],0.00
886,898,2025-01-20 17:48:24,2025-01-20 17:58:40,0,*******,100,615,1,2025-01-20 17:58:40,R_7mOTpZlUsyFQ3pU,...,3.5,-0.5,4.0,beluga,control_personal,Personal Story + Character,control,"[1, 5, 11, 13]",[],4.00
887,899,2025-01-20 17:45:14,2025-01-20 17:59:34,0,*******,100,860,1,2025-01-20 17:59:35,R_6scaMGk4KRE5hFR,...,5.0,0.0,2.0,jellyfish,treatment,Chatbot + Character,treatment,"[1, 5, 7, 9, 13]",[],5.00


## Processing and filtering of participants

In [3]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import plotly.express as px
from statannotations.Annotator import Annotator
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.collections import PathCollection

import seaborn as sns
import sys
import scipy
sys.path.append('/work/include/')
import stat_process
import importlib

import random

# Define a function to balance the groups, if needed
def balance_groups(df, group_column='group'):
    """
    Ensures an even number of participants in each group by randomly reducing
    the size of overrepresented groups.
    
    Parameters:
        df (DataFrame): The dataframe to process.
        group_column (str): The column indicating participant groups.
    
    Returns:
        DataFrame: A dataframe with balanced group sizes.
    """
    group_counts = df[group_column].value_counts()
    min_group_size = group_counts.min()  # Determine the smallest group size

    # Balanced dataframe to hold participants
    balanced_df = pd.DataFrame()
    
    # Adjust each group to the minimum size
    for group, count in group_counts.items():
        group_data = df[df[group_column] == group]
        if count > min_group_size:
            # Randomly sample participants to match the smallest group size
            group_data = group_data.sample(min_group_size, random_state=42)
        balanced_df = pd.concat([balanced_df, group_data])
    
    return balanced_df


importlib.reload(stat_process)

# Define experiment constants and validation rules
EXPERIMENT_CONFIG = {
    'target_n_per_condition': 100,
    'total_target_n': 300,
    'min_completion_time_seconds': 180,  # 3 minutes minimum
}

# Create validation functions
def validate_completion_time(df):
    """Check if completion time meets minimum threshold"""
    duration = pd.to_numeric(df['Duration (in seconds)'], errors='coerce')
    return duration >= EXPERIMENT_CONFIG['min_completion_time_seconds']

def validate_attention_checks(df):
    """Verify attention checks are passed"""
    pre_check = pd.to_numeric(df['attention_pre'], errors='coerce') == 1
    post_check = pd.to_numeric(df['attention_post'], errors='coerce') == 4
    return pre_check & post_check

def validate_response_patterns(df):
    """Check for straight-lining and other suspicious patterns"""
    # Add specific pattern checks based on your survey structure.
    return pd.Series([True] * len(df))  # Currently, simply pass all participants

# --- Outlier Detection Functions ---

def detect_outliers(df, variables, z_threshold=3):
    """
    Returns a boolean Series with True for rows that are NOT outliers.
    A row is flagged as an outlier in a given column if its z-score exceeds z_threshold.
    The function requires the variables list to consist of numeric columns.
    """
    valid = pd.Series(True, index=df.index)
    for var in variables:
        if var in df.columns and pd.api.types.is_numeric_dtype(df[var]):
            z = np.abs((df[var] - df[var].mean()) / df[var].std())
            valid &= (z <= z_threshold)
    return valid

# # List of key variables for outlier detection on 0 to 100 scale.
# outlier_vars = [
#     'belief_pre_score', 'belief_post_score',
#     'distance_pre_score', 'distance_post_score',
#     'policy_pre_score', 'policy_post_score',
#     'adoption_pre_score', 'adoption_post_score'
# ]

# Create a mask for technical issues
def validate_technical_issues(df):
    """Exclude participants with reported technical issues."""
    return df['issues'] != 1

# Create exclusion tracking (now including outlier count)
def track_exclusions(df):
    """Track reasons for excluding participants, including technical issues."""
    failed_time = (~validate_completion_time(df)).sum()
    failed_attention = (~validate_attention_checks(df)).sum()
    failed_patterns = (~validate_response_patterns(df)).sum()
    # failed_outliers = (~detect_outliers(df, outlier_vars, z_threshold=3)).sum()
    failed_technical_issues = (df['issues'] == 1).sum()  # Count participants with technical issues
    
    exclusions = {
        'total_responses': len(df),
        'failed_completion_time': failed_time,
        'failed_attention_check': failed_attention,
        'suspicious_patterns': failed_patterns,
        # 'failed_outlier_check': failed_outliers,
        'technical_issues': failed_technical_issues,  # Log technical issues
        'total_excluded': failed_time + failed_attention + failed_patterns + failed_technical_issues # Readd failed_outliers
    }
    exclusions['final_sample'] = exclusions['total_responses'] - exclusions['total_excluded']
    return pd.Series(exclusions)

# Monitor condition assignment
def monitor_condition_balance(df):
    """Track participant counts per condition"""
    condition_counts = df['group'].value_counts()
    target_n = EXPERIMENT_CONFIG['target_n_per_condition']
    
    balance_status = pd.DataFrame({
        'current_n': condition_counts,
        'target_n': pd.Series([target_n] * len(condition_counts), index=condition_counts.index),
        'remaining_n': pd.Series([target_n] * len(condition_counts), index=condition_counts.index) - condition_counts
    })
    return balance_status

# Print exclusion summary BEFORE removal of participants:
print("\nExclusion Summary (Before Filtering):" + "\n")
print(track_exclusions(df))

# Remove exclusions from dataframe:
# Here we combine all validation criteria, including outlier removal.
valid_mask = (validate_completion_time(df) &
              validate_attention_checks(df) &
              validate_response_patterns(df) &
              # detect_outliers(df, outlier_vars, z_threshold=3) &
              validate_technical_issues(df))  # Exclude participants with technical issues)
df = df[valid_mask]

# If you need to balance the groups, do it here:
# df = balance_groups(df)
print("\nCondition Balance (After Filtering):")
print(monitor_condition_balance(df))


Exclusion Summary (Before Filtering):

total_responses           889
failed_completion_time      1
failed_attention_check      5
suspicious_patterns         0
technical_issues           40
total_excluded             46
final_sample              843
dtype: int64

Condition Balance (After Filtering):
                              current_n  target_n  remaining_n
control_personal_beluga             103       100           -3
treatment_seahorse                   96       100            4
control_personal_jellyfish           96       100            4
control_scientific_jellyfish         95       100            5
control_personal_seahorse            94       100            6
treatment_jellyfish                  92       100            8
control_scientific_beluga            90       100           10
control_scientific_seahorse          90       100           10
treatment_beluga                     87       100           13


In [4]:
# Save Processed data to .csv

df.to_csv("//work/processed/processed_data.csv")

df

,Unnamed: 0,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,...,intention_post_score,intention_change,single_use_perception,animal,group_category,group_label,condition,sustainable_choices,non_sustainable_choices,sustainable_choices_ratio
0,0,2025-01-20 01:33:39,2025-01-20 02:08:14,0,*******,100,2075,1,2025-01-20 02:08:15,R_7ZNWQQgwCOZcotd,...,5.0,0.0,2.0,seahorse,treatment,Chatbot + Character,treatment,"[1, 5, 7, 9, 11, 13]",[],6.00
1,1,2025-01-20 12:23:57,2025-01-20 12:29:57,0,*******,100,360,1,2025-01-20 12:29:58,R_63dfdErmTnYP9rX,...,3.0,-1.0,3.5,beluga,treatment,Chatbot + Character,treatment,[7],[4],1.00
2,2,2025-01-20 15:19:53,2025-01-20 15:23:09,0,*******,100,196,1,2025-01-20 15:23:10,R_6EI5znfDhCu9O0N,...,3.0,0.0,2.0,beluga,control_scientific,Scientific Fact + Image,control,[1],[],1.00
3,3,2025-01-20 15:18:37,2025-01-20 15:23:19,0,*******,100,282,1,2025-01-20 15:23:20,R_6FIR6EYvvCWrp7P,...,3.5,0.0,3.0,jellyfish,control_scientific,Scientific Fact + Image,control,[5],[],1.00
4,4,2025-01-20 15:19:16,2025-01-20 15:23:29,0,*******,100,252,1,2025-01-20 15:23:30,R_773GBdeWwBXKBzX,...,4.0,0.0,3.0,beluga,control_personal,Personal Story + Character,control,[13],[],1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,896,2025-01-20 17:46:16,2025-01-20 17:52:24,0,*******,100,367,1,2025-01-20 17:52:24,R_5uR1Le4uABzActq,...,3.0,0.0,5.0,seahorse,control_personal,Personal Story + Character,control,[5],[],1.00
885,897,2025-01-20 17:40:42,2025-01-20 17:53:28,0,*******,100,766,1,2025-01-20 17:53:29,R_1BXf09kTVDpJklz,...,2.5,0.0,2.0,seahorse,treatment,Chatbot + Character,treatment,[],[4],0.00
886,898,2025-01-20 17:48:24,2025-01-20 17:58:40,0,*******,100,615,1,2025-01-20 17:58:40,R_7mOTpZlUsyFQ3pU,...,3.5,-0.5,4.0,beluga,control_personal,Personal Story + Character,control,"[1, 5, 11, 13]",[],4.00
887,899,2025-01-20 17:45:14,2025-01-20 17:59:34,0,*******,100,860,1,2025-01-20 17:59:35,R_6scaMGk4KRE5hFR,...,5.0,0.0,2.0,jellyfish,treatment,Chatbot + Character,treatment,"[1, 5, 7, 9, 13]",[],5.00


## Process Outcome Vars for Regression

In [7]:
import pandas as pd
import numpy as np
import ast

############################
# CREATE NEW DF
############################

def safe_literal_eval(val):
    """Safely convert a string to a Python literal (list) or return an empty list on failure."""
    try:
        if isinstance(val, str):
            return ast.literal_eval(val)
        elif isinstance(val, list):
            return val
        else:
            return []
    except (ValueError, SyntaxError):
        return []

# -- Climate Change Belief (pre/post/change) --
df['climate_change_belief_pre'] = df[['belief_pre_1', 'belief_pre_2', 'belief_pre_3', 'belief_pre_4']].mean(axis=1)
df['climate_change_belief_post'] = df[['belief_post_1', 'belief_post_2', 'belief_post_3', 'belief_post_4']].mean(axis=1)
df['climate_change_belief_change'] = df['climate_change_belief_post'] - df['climate_change_belief_pre']

# -- Psychological Distance (Overall) --
df['psychological_distance_towards_climate_change_overall_pre'] = df[['distance_pre_1','distance_pre_2','distance_pre_3','distance_pre_4','distance_pre_5']].mean(axis=1)
df['psychological_distance_towards_climate_change_overall_post'] = df[['distance_post_1','distance_post_2','distance_post_3','distance_post_4','distance_post_5']].mean(axis=1)
df['psychological_distance_towards_climate_change_overall_change'] = (
    df['psychological_distance_towards_climate_change_overall_post']
    - df['psychological_distance_towards_climate_change_overall_pre']
)


# -- Psychological Distance (Spatial) --
df['psychological_distance_towards_climate_change_spatial_pre'] = df[['distance_pre_1','distance_pre_2','distance_pre_3']].mean(axis=1)
df['psychological_distance_towards_climate_change_spatial_post'] = df[['distance_post_1','distance_post_2','distance_post_3',]].mean(axis=1)
df['psychological_distance_towards_climate_change_spatial_change'] = (
    df['psychological_distance_towards_climate_change_spatial_post']
    - df['psychological_distance_towards_climate_change_spatial_pre']
)


# -- Psychological Distance (Temporal) --
df['psychological_distance_towards_climate_change_temporal_pre'] = df[['distance_pre_4','distance_pre_5']].mean(axis=1)
df['psychological_distance_towards_climate_change_temporal_post'] = df[['distance_post_4','distance_post_5']].mean(axis=1)
df['psychological_distance_towards_climate_change_temporal_change'] = (
    df['psychological_distance_towards_climate_change_temporal_post']
    - df['psychological_distance_towards_climate_change_temporal_pre']
)

# -- Climate Policy Adoption (pre/post/change) --
df['climate_policy_adoption_pre'] = df[['policy_pre_1','policy_pre_2','policy_pre_3','policy_pre_4','policy_pre_5','policy_pre_6','policy_pre_7']].mean(axis=1)
df['climate_policy_adoption_post'] = df[['policy_post_1','policy_post_2','policy_post_3','policy_post_4','policy_post_5','policy_post_6','policy_post_7']].mean(axis=1)
df['climate_policy_adoption_change'] = df['climate_policy_adoption_post'] - df['climate_policy_adoption_pre']

# -- Sustainable Consumption (Adoption) (pre/post/change) --
df['sustainable_consumption_adoption_pre'] = df[['adoption_pre_1','adoption_pre_2','adoption_pre_3','adoption_pre_4','adoption_pre_5','adoption_pre_6']].mean(axis=1)
df['sustainable_consumption_adoption_post'] = df[['adoption_post_1','adoption_post_2','adoption_post_3','adoption_post_4','adoption_post_5','adoption_post_6']].mean(axis=1)
df['sustainable_consumption_adoption_change'] = (
    df['sustainable_consumption_adoption_post']
    - df['sustainable_consumption_adoption_pre']
)

# -- Willingness to Share Information (pre/post/change) --
df['willingness_to_share_information_pre'] = df['sharing_pre'].apply(lambda x: 1 if x == 1 else (0 if x == 2 else np.nan))
df['willingness_to_share_information_post'] = df['sharing_post'].apply(lambda x: 1 if x == 1 else (0 if x == 2 else np.nan))
df['willingness_to_share_information_change'] = (
    df['willingness_to_share_information_post']
    - df['willingness_to_share_information_pre']
)

# -- Intervention Sharing (0/1) --

df['intervention_sharing_post'] = df['sharingwilling_post'].apply(
    lambda x: 1 if x == 1 else (0 if x == 2 else np.nan)
)

#############################
# Anthropomorphism (1 item)
#############################
df['perceived_anthropomorphism_post'] = df['anthropomorphism_1']

#############################
# Animacy (2 items)
#############################
df['perceived_animacy_post'] = df[['animacy_1', 'animacy_2']].mean(axis=1)


#############################
# Likeability (1 item)
#############################
df['perceived_likeability_post'] = df['likeability_1']


#############################
# Perceived Intelligence (2 items)
#############################
df['perceived_intelligence_post'] = df[['perceived_intel_1', 'perceived_intel_2']].mean(axis=1)


#############################
# Perceived Safety (1 item)
#############################
df['perceived_safety_post'] = df['perceived_safety_1']

#############################
# Empathy with the Animal (1 item)
#############################
df['empathy_with_the_animal_post'] = df['empathy_with_animal_1']


#############################
# Climate Change (1 item)
#############################
df['climate_change_impact_on_animal_post'] = df['climate_impact_1']


#############################
# Pro-environmental Intentions & Behavior
#############################
df['pro_environmental_intentions_pre'] = df[['single_use2_pre', 'refillable_pre']].mean(axis=1)
df['pro_environmental_intentions_post'] = df[['single_use2_post', 'refillable_post']].mean(axis=1)
df['pro_environmental_intentions_change'] = (
    df['pro_environmental_intentions_post'] 
    - df['pro_environmental_intentions_pre']
)

# For example, a single-use measure before vs after
df['pro_environmental_behavior_pre'] = df[['single_use_pre', 'single_use_post']].mean(axis=1)
df['pro_environmental_behavior_post'] = df[['single_use_post', 'single_use_post']].mean(axis=1)
df['pro_environmental_behavior_change'] = (
    df['pro_environmental_behavior_post'] 
    - df['pro_environmental_behavior_pre']
)

#############################
# Convert & Calculate Sustainable Choice Preferences
#############################
df['behavior_post'] = df['behavior_post'].apply(safe_literal_eval)
df['behavior_post'] = df['behavior_post'].apply(lambda x: [int(item) for item in x])

sustainable_products = [1, 5, 7, 9, 11, 13]
non_sustainable_products = [4, 6, 8, 10, 12, 14]

df['sustainable_choices'] = df['behavior_post'].apply(lambda x: [p for p in x if p in sustainable_products])
df['non_sustainable_choices'] = df['behavior_post'].apply(lambda x: [p for p in x if p in non_sustainable_products])

def custom_ratio(sustainable_list, non_sustainable_list):
    """Compute a ratio or custom metric for sustainable vs non-sustainable products."""
    count_sust = len(sustainable_list)
    count_nonsust = len(non_sustainable_list)
    
    if count_sust > 0 and count_nonsust > 0:
        return count_sust / count_nonsust
    elif count_sust > 0 and count_nonsust == 0:
        return count_sust  # or 1.0, or any special rule
    elif count_sust == 0 and count_nonsust > 0:
        return 0  # or 0.0, or any special rule
    else:
        return np.nan

df['sustainable_choice_preferences'] = df.apply(
    lambda row: custom_ratio(row['sustainable_choices'], row['non_sustainable_choices']),
    axis=1
)

#############################
# Experimental Condition & Animal
#############################
def map_experimental_condition(group_name):
    """
    Returns the experimental condition label based on substring matching.
    """
    if isinstance(group_name, str):
        if "control_scientific" in group_name:
            return "Static Scientific Information"
        elif "control_personal" in group_name:
            return "Static Character Narrative"
        elif "treatment" in group_name:
            return "Conversational Character Narrative"
    return "Unknown"

df['experimental_condition'] = df['group'].apply(map_experimental_condition)

animals = ["seahorse", "beluga", "jellyfish"]
def extract_animal(group_name):
    for animal in animals:
        if animal in group_name:
            return animal
    return "Unknown"

df['animal'] = df['group'].apply(extract_animal)

#############################
# Final Subsetting
#############################
# List *all* newly created columns that appear in the script:
newly_created_cols = [
    "climate_change_belief_pre",
    "climate_change_belief_post",
    "climate_change_belief_change",
    "psychological_distance_towards_climate_change_overall_pre",
    "psychological_distance_towards_climate_change_overall_post",
    "psychological_distance_towards_climate_change_overall_change",
    "psychological_distance_towards_climate_change_spatial_pre",
    "psychological_distance_towards_climate_change_spatial_post",
    "psychological_distance_towards_climate_change_spatial_change",
    "psychological_distance_towards_climate_change_temporal_pre",
    "psychological_distance_towards_climate_change_temporal_post",
    "psychological_distance_towards_climate_change_temporal_change",
    "climate_policy_adoption_pre",
    "climate_policy_adoption_post",
    "climate_policy_adoption_change",
    "sustainable_consumption_adoption_pre",
    "sustainable_consumption_adoption_post",
    "sustainable_consumption_adoption_change",
    "willingness_to_share_information_pre",
    "willingness_to_share_information_post",
    "willingness_to_share_information_change",
    "intervention_sharing_pre",
    "intervention_sharing_post",
    "intervention_sharing_change",
    "perceived_anthropomorphism_pre",
    "perceived_anthropomorphism_post",
    "perceived_anthropomorphism_change",
    "perceived_animacy_pre",
    "perceived_animacy_post",
    "perceived_animacy_change",
    "perceived_likeability_pre",
    "perceived_likeability_post",
    "perceived_likeability_change",
    "perceived_intelligence_pre",
    "perceived_intelligence_post",
    "perceived_intelligence_change",
    "perceived_safety_pre",
    "perceived_safety_post",
    "perceived_safety_change",
    "empathy_with_the_animal_pre",
    "empathy_with_the_animal_post",
    "empathy_with_the_animal_change",
    "climate_change_impact_on_animal_pre",
    "climate_change_impact_on_animal_post",
    "climate_change_impact_on_animal_change",
    "pro_environmental_intentions_pre",
    "pro_environmental_intentions_post",
    "pro_environmental_intentions_change",
    "pro_environmental_behavior_pre",
    "pro_environmental_behavior_post",
    "pro_environmental_behavior_change",
    "sustainable_choice_preferences",
    "experimental_condition",
    "animal"
]

# Plus any personal/demographic columns you wish to preserve:
demographics = [
    "participantId",
    "age",
    "gender",
    "education",
    "location"
]

# Combine them to form your final column set:
columns_to_keep = demographics + newly_created_cols

# Retain only those columns in the DataFrame that actually exist:
columns_to_keep = [col for col in columns_to_keep if col in df.columns]

# Subset the DataFrame
export_df = df[columns_to_keep]

# (Optional) Inspect final columns
print("Final columns:\n", export_df.columns.tolist())

export_df

Final columns:
 ['participantId', 'age', 'gender', 'education', 'location', 'climate_change_belief_pre', 'climate_change_belief_post', 'climate_change_belief_change', 'psychological_distance_towards_climate_change_overall_pre', 'psychological_distance_towards_climate_change_overall_post', 'psychological_distance_towards_climate_change_overall_change', 'psychological_distance_towards_climate_change_spatial_pre', 'psychological_distance_towards_climate_change_spatial_post', 'psychological_distance_towards_climate_change_spatial_change', 'psychological_distance_towards_climate_change_temporal_pre', 'psychological_distance_towards_climate_change_temporal_post', 'psychological_distance_towards_climate_change_temporal_change', 'climate_policy_adoption_pre', 'climate_policy_adoption_post', 'climate_policy_adoption_change', 'sustainable_consumption_adoption_pre', 'sustainable_consumption_adoption_post', 'sustainable_consumption_adoption_change', 'willingness_to_share_information_pre', 'willing

,participantId,age,gender,education,location,climate_change_belief_pre,climate_change_belief_post,climate_change_belief_change,psychological_distance_towards_climate_change_overall_pre,psychological_distance_towards_climate_change_overall_post,...,climate_change_impact_on_animal_post,pro_environmental_intentions_pre,pro_environmental_intentions_post,pro_environmental_intentions_change,pro_environmental_behavior_pre,pro_environmental_behavior_post,pro_environmental_behavior_change,sustainable_choice_preferences,experimental_condition,animal
0,985A197E95D5425BA2DC169134A42F35,46.0,2.0,4.0,2.0,100.00,100.00,0.00,4.8,5.0,...,5.0,5.0,5.0,0.0,2.0,2.0,0.0,6.00,Conversational Character Narrative,seahorse
1,NaN,NaN,1.0,NaN,2.0,1.00,1.75,0.75,2.0,2.8,...,3.0,4.0,3.0,-1.0,3.5,4.0,0.5,1.00,Conversational Character Narrative,beluga
2,853B2FE41C89428FA5E51DEB9AAAD08A,33.0,1.0,3.0,2.0,68.75,66.75,-2.00,4.4,4.0,...,5.0,3.0,3.0,0.0,2.0,2.0,0.0,1.00,Static Scientific Information,beluga
3,5C342B0A6CCF40929D2EA3DA67248313,21.0,2.0,4.0,2.0,93.50,100.00,6.50,6.0,7.0,...,5.0,3.5,3.5,0.0,3.0,3.0,0.0,1.00,Static Scientific Information,jellyfish
4,EAE68CF98F144BE29A3046AA3C8BA239,42.0,1.0,7.0,1.0,100.00,100.00,0.00,4.0,4.0,...,5.0,4.0,4.0,0.0,3.0,3.0,0.0,1.00,Static Character Narrative,beluga
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884,246C71A50D8A495881CF078470F12D6A,34.0,2.0,3.0,3.0,39.50,56.25,16.75,4.4,4.4,...,4.0,3.0,3.0,0.0,5.0,5.0,0.0,1.00,Static Character Narrative,seahorse
885,65AC515D82704F11BCCA446DD239C56E,25.0,1.0,5.0,3.0,41.50,37.75,-3.75,4.4,4.4,...,5.0,2.5,2.5,0.0,2.0,2.0,0.0,0.00,Conversational Character Narrative,seahorse
886,AD4EC7E7D2194A7FBC94F4966EA2D556,25.0,1.0,2.0,3.0,77.00,78.25,1.25,5.2,4.6,...,5.0,4.0,3.5,-0.5,4.0,4.0,0.0,4.00,Static Character Narrative,beluga
887,AADA39CC0AA64DFB84C73828D0AEBDFB,46.0,2.0,3.0,2.0,100.00,100.00,0.00,4.6,4.0,...,5.0,5.0,5.0,0.0,2.0,2.0,0.0,5.00,Conversational Character Narrative,jellyfish


## Cronbachs

In [8]:
import pandas as pd
import numpy as np
import re

# =============================================================================
# 1. Load and Inspect Data
# =============================================================================

# Load your DataFrame (replace 'your_data.csv' with your actual data source)
# If your DataFrame is already loaded as 'df', you can skip this step
# Example:
# df = pd.read_csv('your_data.csv')

# Here, assuming 'df' is already defined in your environment
data = df.copy()

# Display initial data shape
print("Initial Data Shape:", data.shape)

# Display first few rows to understand the structure
print("\nFirst Few Rows of Original Data:")
print(data.head())

# =============================================================================
# 2. Identify Questionnaire Columns
# =============================================================================

# Define columns to exclude (non-questionnaire columns)
exclude_columns = [
    'Unnamed: 0', 'StartDate', 'EndDate', 'Status', 'IPAddress',
    'Progress', 'Duration (in seconds)', 'Finished', 'RecordedDate',
    'ResponseId', 'RecipientLastName', 'RecipientFirstName',
    'RecipientEmail', 'ExternalReference', 'LocationLatitude',
    'LocationLongitude', 'DistributionChannel', 'UserLanguage',
    'Q_RecaptchaScore', 'Intro',
    'Intervention Time_First Click', 'Intervention Time_Last Click',
    'Intervention Time_Page Submit', 'Intervention Time_Click Count',
    'age', 'gender', 'education',
    # Add any other non-questionnaire columns here
]

# Identify questionnaire columns
questionnaire_columns = [col for col in data.columns if col not in exclude_columns]

print("\nQuestionnaire Columns Identified:")
print(questionnaire_columns)

# =============================================================================
# 3. Extract Base Names and Group into Subscales
# =============================================================================

# Function to extract base name by removing trailing '_number' if present
def extract_base_name(column_name):
    """
    Extracts the base name by removing trailing '_number'.
    If no trailing number, returns the original column name.
    """
    match = re.match(r"^(.*?)(?:_\d+)?$", column_name)
    if match:
        return match.group(1)
    else:
        return column_name

# Apply the function to all questionnaire columns to get base names
base_names = pd.Series(questionnaire_columns).apply(extract_base_name)

# Create a DataFrame mapping columns to base names
base_names_df = pd.DataFrame({
    'column': questionnaire_columns,
    'base_name': base_names
})

# Display the mapping
print("\nColumn to Base Name Mapping:")
print(base_names_df.head(10))

# Group columns by base_name
subscale_groups = base_names_df.groupby('base_name')['column'].apply(list)

# Convert to dictionary
subscale_dict = subscale_groups.to_dict()

# Print all subscales and their items
print("\nAll Identified Subscales:")
for subscale, items in subscale_dict.items():
    print(f"{subscale}: {items}")

# =============================================================================
# 4. Filter Valid Subscales
# =============================================================================

# Define minimum number of items for a valid subscale
min_items = 2

# Filter subscales to include only those with at least 'min_items'
valid_subscales = {k: v for k, v in subscale_dict.items() if len(v) >= min_items}

print("\nValid Subscales (with at least 2 items):")
for subscale, items in valid_subscales.items():
    print(f"{subscale}: {items}")

# =============================================================================
# 5. Handle Missing Data Appropriately
# =============================================================================

# Handle missing data

# Option 1: Impute numeric columns with mean
numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()
data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

# Option 2: Impute categorical columns with mode
categorical_cols = data.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_cols:
    if col in data.columns:
        data[col].fillna(data[col].mode()[0], inplace=True)

# Verify that there are no missing values left
total_missing = data.isnull().sum().sum()
print("\nTotal Missing Values After Imputation:", total_missing)

# =============================================================================
# 6. Calculate Cronbach’s Alpha for Each Subscale
# =============================================================================

# Define Cronbach's Alpha function
def cronbach_alpha(df):
    """
    Calculates Cronbach's Alpha for a given DataFrame of items.
    
    Parameters:
    - df: pandas DataFrame containing the items.
    
    Returns:
    - Cronbach's Alpha as a float.
    """
    N = df.shape[1]
    
    if N < 2:
        print("Cronbach's alpha is not defined for less than two items.")
        return np.nan
    
    variances = df.var(ddof=1)
    total_scores = df.sum(axis=1)
    total_variance = total_scores.var(ddof=1)
    
    if total_variance == 0:
        print("Total variance is zero. Cronbach's alpha is undefined.")
        return np.nan
    
    alpha = (N / (N - 1)) * (1 - (variances.sum() / total_variance))
    return alpha

# Function to interpret Cronbach's Alpha
def interpret_alpha(alpha):
    """
    Provides an interpretation of Cronbach's Alpha value.
    
    Parameters:
    - alpha: Cronbach's Alpha value.
    
    Returns:
    - Interpretation string.
    """
    if np.isnan(alpha):
        return "Undefined"
    elif alpha >= 0.9:
        return "Excellent"
    elif alpha >= 0.8:
        return "Good"
    elif alpha >= 0.7:
        return "Acceptable"
    elif alpha >= 0.6:
        return "Questionable"
    elif alpha >= 0.5:
        return "Poor"
    else:
        return "Unacceptable"

# Calculate Cronbach's Alpha for each valid subscale
alpha_results = {}

for subscale, items in valid_subscales.items():
    # Select subscale data
    subscale_data = data[items]
    
    # Ensure all items are numeric
    non_numeric = subscale_data.select_dtypes(exclude=[np.number]).columns.tolist()
    if non_numeric:
        print(f"\nSubscale '{subscale}' has non-numeric items: {non_numeric}. Skipping...")
        alpha_results[subscale] = (np.nan, "Non-Numeric Items")
        continue
    
    # Calculate Cronbach's Alpha
    alpha = cronbach_alpha(subscale_data)
    
    # Interpret the alpha value
    interpretation = interpret_alpha(alpha)
    
    # Store the result
    alpha_results[subscale] = (alpha, interpretation)

# Display the results
print("\nCronbach's Alpha Results:")
for subscale, (alpha, interpretation) in alpha_results.items():
    print(f"- {subscale}: {alpha:.3f} ({interpretation})")

Initial Data Shape: (843, 171)

First Few Rows of Original Data:
   Unnamed: 0            StartDate              EndDate  Status IPAddress  \
0           0  2025-01-20 01:33:39  2025-01-20 02:08:14       0   *******   
1           1  2025-01-20 12:23:57  2025-01-20 12:29:57       0   *******   
2           2  2025-01-20 15:19:53  2025-01-20 15:23:09       0   *******   
3           3  2025-01-20 15:18:37  2025-01-20 15:23:19       0   *******   
4           4  2025-01-20 15:19:16  2025-01-20 15:23:29       0   *******   

   Progress  Duration (in seconds)  Finished         RecordedDate  \
0       100                   2075         1  2025-01-20 02:08:15   
1       100                    360         1  2025-01-20 12:29:58   
2       100                    196         1  2025-01-20 15:23:10   
3       100                    282         1  2025-01-20 15:23:20   
4       100                    252         1  2025-01-20 15:23:30   

          ResponseId  ... empathy_with_the_animal_post  \

TypeError: "value" parameter must be a scalar or dict, but you passed a "list"

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1013c793-54a5-4345-8056-59dcfc0f7c3d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>